In [29]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
from gensim import corpora
from gensim import models
from gensim import similarities
from gensim.models import Word2Vec
from gensim.models.coherencemodel import CoherenceModel

In [30]:
declaracoes = pd.read_csv("declaracoes-nlp-no-punct.csv", usecols=["speaker", "partido", "filename", "no_punct"])
declaracoes

,speaker,partido,filename,no_punct
0,Emília Cerqueira,PSD,darl13sl03n050.txt,sr presidente srs deputados saúdo os verdes po...
1,Emília Cerqueira,PSD,darl13sl03n050.txt,portanto estamos a falar infelizmente de uma s...
2,Emília Cerqueira,PSD,darl13sl03n050.txt,terminarei sr presidente perguntando a os verd...
3,Emília Cerqueira,PSD,darl13sl03n050.txt,não é verdade
4,Emília Cerqueira,PSD,darl13sl03n050.txt,sr presidente sr deputado pedro soares hoje te...
...,...,...,...,...
92570,Heloísa Apolónia,PEV,darl13sl03n058.txt,sr presidente sras e srs deputados o problema ...
92571,Heloísa Apolónia,PEV,darl13sl03n058.txt,é por estas razões e por tantas outras sr pres...
92572,António Filipe,PCP,darl13sl03n058.txt,sr presidente srª ministra srª secretária de e...
92573,António Filipe,PCP,darl13sl03n058.txt,sr presidente srs membros do governo srs deput...


In [31]:
documents = list(declaracoes["no_punct"])
documents = [str(x) for x in documents]
documents = [x.replace(r"\d*",'') for x in documents]

# remove common words and tokenize
stoplist = set('de que e o do da é para em não os um uma com no as dos sr na se mais por das ao a ou também são portanto muito como à às sobre outro outra outros outras nos nas srª'.split())
texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [32]:
#LDA

#tfidf_corpus = models.TfidfModel(corpus, normalize=True)

lda_model = models.LdaModel(corpus, id2word=dictionary, num_topics=50)

In [33]:
coherence_model_lda = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  -2.8622370409085027


In [34]:
topics_df = pd.DataFrame(lda_model.print_topics())
topics_df

,0,1
0,46,"0.044*""caixa"" + 0.043*""geral"" + 0.040*""comissã..."
1,11,"0.223*""presidente"" + 0.203*""palavra"" + 0.120*""..."
2,39,"0.151*""deputada"" + 0.085*""partido"" + 0.054*""so..."
3,3,"0.084*""pcp"" + 0.031*""medidas"" + 0.018*""problem..."
4,30,"0.263*""esquerda"" + 0.256*""bloco"" + 0.066*""bem""..."
5,17,"0.052*""trabalho"" + 0.051*""trabalhadores"" + 0.0..."
6,19,"0.045*""quatro"" + 0.039*""jogo"" + 0.035*""algarve..."
7,26,"0.100*""mínimo"" + 0.090*""salário"" + 0.054*""aume..."
8,25,"0.113*""presidente"" + 0.080*""voto"" + 0.056*""dec..."
9,47,"0.014*""nacionalidade"" + 0.014*""governo"" + 0.01..."


In [35]:
lda_model.print_topics()

[(31,
  '0.116*"estado" + 0.053*"justiça" + 0.036*"secretário" + 0.020*"direito" + 0.020*"administração" + 0.015*"tribunais" + 0.014*"funcionários" + 0.012*"secretária" + 0.011*"cidadãos" + 0.011*"processos"'),
 (20,
  '0.063*"era" + 0.046*"antónio" + 0.037*"vosso" + 0.034*"costa" + 0.019*"vocês" + 0.018*"vossa" + 0.016*"memória" + 0.015*"psd" + 0.014*"foi" + 0.014*"filipe"'),
 (39,
  '0.151*"deputada" + 0.085*"partido" + 0.054*"socialista" + 0.036*"pcp" + 0.022*"protestos" + 0.019*"comunista" + 0.014*"português" + 0.013*"propostas" + 0.011*"verdes" + 0.010*"aqui"'),
 (36,
  '0.040*"2018" + 0.034*"setor" + 0.033*"2017" + 0.030*"2019" + 0.025*"2016" + 0.023*"2020" + 0.023*"1" + 0.023*"maio" + 0.023*"7" + 0.021*"5"'),
 (47,
  '0.014*"nacionalidade" + 0.014*"governo" + 0.011*"andré" + 0.010*"falhou" + 0.010*"srs" + 0.009*"apoiam" + 0.009*"deputados" + 0.007*"culpa" + 0.007*"chega" + 0.007*"minutos"'),
 (9,
  '0.017*"cultura" + 0.015*"governo" + 0.011*"património" + 0.010*"srs" + 0.010*"ce